# API

> Implementation of fastgit

In [ ]:
#| default_exp core

### Imports

In [ ]:
#| export
from fastcore.utils import *
import subprocess
from subprocess import CalledProcessError
import shutil,stat

In [ ]:
import tempfile
from fastcore.test import test_eq

In [ ]:
#| export
def callgit(path, *args, split=None, uname=None):
    fp = Path(path).resolve()
    args = ['git', '-C', str(fp)] + list(args)
    if uname: args = ['/usr/bin/sudo', '-u', uname] + args
    res = subprocess.run(args, capture_output=True, text=True, check=True).stdout.strip()
    if split is None: return res.splitlines() if '\n' in res else res
    return res.splitlines() if split else res

In [ ]:
#| export
def get_top(folder):
    try: return callgit(folder, 'rev-parse', '--show-toplevel')
    except CalledProcessError: return None

In [ ]:
with tempfile.TemporaryDirectory() as td:
    test_eq(get_top(td), None)
    msg = callgit(td, 'init')
    assert get_top(td) in msg

`Git` represents a python wrapper for the `git` command. It will execute every command as if `git` were being run with the working directory being the directory `d` passed in at init time.

As a result, callers should take care not to checkout a branch which does not have the directory used to initialize `Git`.

In [ ]:
#| export
class Git:
    def __init__(self, d): self.d = Path(d)

    def __call__(self, cmd, *args, split=None, mute_errors=False, **kwargs):
        args = listify(args)
        args += concat((f'-{k}',v) for k,v in kwargs.items() if len(k)==1 and v is not True)
        args += [f'-{k}' for k,v in kwargs.items() if len(k)==1 and v is True]
        args += [f'--{k.replace("_","-")}={v}' for k,v in kwargs.items() if len(k)>1 and v is not True and v is not False]
        args += [f'--{k.replace("_","-")}' for k,v in kwargs.items() if len(k)>1 and v is True]
        try: return callgit(self.d, cmd, *args, split=split)
        except CalledProcessError as e:
            if not mute_errors: print(f'ERROR: Git.__call__ caught exception {e} \n with stderr={e.stderr}')

    def __getattr__(self, nm):
        if nm.startswith('_'): raise AttributeError(nm)
        return partial(self, nm.replace('_','-'))

    def top(self): return self.rev_parse('--show-toplevel', mute_errors=True)
    
    @property
    def exists(self): return self.top() is not None

In [ ]:
with tempfile.TemporaryDirectory() as td:
    g = Git(td)
    assert not g.exists
    msg = g.init()
    assert g.top() in msg

In [ ]:
#| export
@patch(as_prop=True)
def last_commit(self:Git): return self.log('-1', pretty='format:%s')

## export -

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()